## Ocean Wave Data For Stations Recorded by National Data Buoy Center (NDBC)

## Table of Contents

1. [**Importing Necessary Libraries**](#0)

2. [**Data Collection**](#10)

3. [**A look at the Data**](#20)


## 1. Importing Necessary Libraries <a class="anchor" id="0"></a>

In [ ]:
# Let's install and import the required libraries
# Uncomments the following lines in case you don't have the libraries installed on your machine
#!conda install -c conda-forge beautifulsoup4 --yes 
#!conda install -c conda-forge lxml --yes
#!conda install -c conda-forge requests --yes
#!conda install -c conda-forge folium=0.9 --yes
#!conda install -c conda-forge windrose=1.6.7 --yes

import sys
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen, Request
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from windrose import WindroseAxes
import folium
import json
from datetime import date, timedelta, datetime
# Put figures in the center
from IPython.core.display import HTML
import seaborn as sns
sns.set(style="whitegrid")
print('All libraries installed and imported')

## 2. Data Collection <a class="anchor" id="10"></a>

Now, let's get the list of all the buoys from National Data Buoy Center. To get access to the data, we need to first create an account [here](https://data.planetos.com/datasets/noaa_ndbc_stdmet_stations?utm_source=github&utm_medium=notebook&utm_campaign=ndbc-wavewatch-iii-notebook). Once the account is created, API Key can be found under __Account Setting__. The data set id for National Data Buoy Center (NDBC) Standard Meteorological Data is __noaa_ndbc_stdmet_stations__. This data typically updates every hour.

### First, we use an API to get Buys' latitude, longitude and names.

In [ ]:
# @ hidden
APIKey = 
DatasetID = 

In [ ]:
API_url = 'http://api.planetos.com/v1/datasets/%s/stations?apikey=%s' % (DatasetID, APIKey)
request = requests.get(API_url).json() #Request(API_url)
Stations = pd.DataFrame(request['station'])

df_stations = pd.DataFrame(Stations.columns)
for indx, StName in enumerate(Stations.columns):
    df_stations.loc[indx,1]=Stations.loc['SpatialExtent'][indx]['coordinates'][0]
    df_stations.loc[indx,2]=Stations.loc['SpatialExtent'][indx]['coordinates'][1]

df_stations.columns = ['Station_Name','Latitude','Longitude']
print('number of stations:',df_stations.shape[0])
df_stations.head()

### Since the data from this API is not complete for each buoy and also have missing columns, we use the National Data Buoy Center website directly at https://www.ndbc.noaa.gov/ to get our information.

There is a list of stations categorized by their owners [here](https://www.ndbc.noaa.gov/to_station.shtml)

In [ ]:
source=requests.get('https://www.ndbc.noaa.gov/to_station.shtml').text
soup = BeautifulSoup(source,'lxml')
Owners=[]
BuoyData={}

for owner in soup.find_all('h4'):
    Owners.append(owner.text)
for indx,table in enumerate(soup.find_all('pre')):
    #print(indx,table.text)
    try:
        BuoyData[Owners[indx]]=table.text.replace('\n','').strip()
    except:
        BuoyData[Owners[indx]]=table


In [ ]:
# First, let's make all the station names uppercase since they are in lowercase format in the website
df_stations['Station_Name']=df_stations['Station_Name'].str.upper()
# Now, let's put this data and the data containing the latitudes and longitudes in the same dataframe
newlist1=[]
newlist2=[]
for indx1 in range(df_stations.shape[0]):
    for indx2,owner in enumerate(Owners):
        if (df_stations.iloc[indx1,0] in BuoyData[owner]):
            newlist1.append(owner)
            newlist2.append(df_stations.iloc[indx1,0])


In [ ]:
df_stations0=pd.DataFrame({'Station_Name':newlist2,'Owner':newlist1})
df_full1=pd.merge(df_stations0, df_stations, on='Station_Name')
print('Number of Stations:',df_full1.shape[0])
df_full1.head()

In [ ]:
def BuoyLocationPlot2(buoys,MapInput,DataFrameInput,colorm,fill_colorm,Marker=False):
    # instantiate a feature group for the buoys in the dataframe
    
    # loop through and add each to the feature group
    for lat, lng, label in zip(DataFrameInput.Longitude, DataFrameInput.Latitude, DataFrameInput.Station_Name):
        buoys.add_child(
            folium.CircleMarker(
                [lat, lng],
                radius=6, # define how big you want the circle markers to be
                color=colorm,
                fill=True,
                fill_color=fill_colorm,
                fill_opacity=0.8,
                popup=label
            )
        )

    
    if Marker==True:
        latitudes = list(DataFrameInput.Longitude)
        longitudes = list(DataFrameInput.Latitude)
        labels = list(DataFrameInput.Station_Name)
        for lat, lng, label in zip(latitudes, longitudes, labels):
            folium.Marker([lat, lng], popup=label).add_to(MapInput)    
    
    
    return buoys

### Available Data for each buoy
Not all the buys have data available for the same period of time. Some have been working for a short period of time, some had worked before but not any more, and some of them recently started working and collecting data. To have a full collection of the time range of the data available for each buoy, the following page is used from National Data Buoy Center website:
https://www.ndbc.noaa.gov/historical_data.shtml#stdmet. It is worth mentioning that we are only interested in Standard Meterological data. To have a better understanding of  Standard Meterological data, look at the information provided here: https://www.ndbc.noaa.gov/measdes.shtml

In [ ]:
source=requests.get('https://www.ndbc.noaa.gov/historical_data.shtml').text
soup = BeautifulSoup(source,'lxml')

BuoyYearData={}
BigTable=soup.find_all('ul')[1]
SmallTable=BigTable.li.ul

for indx,subtable in enumerate(SmallTable.find_all('li')):
    StatName=SmallTable.find_all('li')[indx].text.split('\n')[0].replace(':','')
    est=SmallTable.find_all('li')[indx].text.split('\n')
    tmplist = est[1:]
    for indx00,value in enumerate(tmplist):
        tmplist[indx00] = tmplist[indx00].strip()
    del tmplist[-1]
    BuoyYearData[StatName]=tmplist
    

# Let's add one column for each year to the main dataframe
YearsRange = range(1970,2019)
df_full = df_full1.copy(deep=True)
df_full.set_index(['Station_Name'],inplace=True)
for indx, year in enumerate(YearsRange):
    df_full[year] = None

for indx,StName in enumerate(df_full.index):
    first=np.array(df_full.loc[StName][4:].index).astype('int64')
    try:
        second=np.array(BuoyYearData[StName]).astype('int64')
        df_full.loc[StName ,np.intersect1d(first,second)]=1
    except:
        df_full.loc[StName ,np.intersect1d(first,second)]=None

In [ ]:
# Plotting the buoys with their corresponding owners
figmp=folium.Figure(width=1300, height=700)
WorldMap_map2 = folium.Map(location=[17.6078, -8.0817],tiles="Stamen Toner",zoom_start=2).add_to(figmp)
color=iter(plt.cm.rainbow(np.linspace(0,1,len(Owners))))
for indx,owner in enumerate(Owners):
    clr=matplotlib.colors.to_hex(next(color))
    feature_group = folium.map.FeatureGroup(name=owner)
    df_tmp = df_full1[df_full1['Owner']==owner]
    buoys=BuoyLocationPlot2(feature_group,WorldMap_map2,df_tmp,clr,clr)
    WorldMap_map2.add_child(buoys,name=owner,index=indx)
    
folium.TileLayer("Stamen Toner").add_to(WorldMap_map2) 
folium.TileLayer("OpenStreetMap").add_to(WorldMap_map2) 
folium.TileLayer("Stamen Terrain").add_to(WorldMap_map2)
WorldMap_map2.add_child(folium.map.LayerControl())
WorldMap_map2.save("WorldMapBuoysOwners.html")
#WorldMap_map2.save("WorldMapBuoysOwners.eps",format='eps') doesn't work!
WorldMap_map2

## 3. A look at the data <a class="anchor" id="20"></a>

#### Get the data for a specific buoy

In [ ]:
# Test Buoy: 46042 at Monterey Bay, California

DumVar="No"
while DumVar=="No":
    BName=input("Enter Buoy's name:")
    try:
        print('Available annual data for this buoy:',BuoyYearData[BName])
    except:
        sys.exit('Error! There is no such buoy!!')
    BYear=input("Year:")
    DumVar="Yes" if ( df_full.loc[BName,int(BYear)]==1 ) else "No"
    print("Is data for buoy {} availabale during year {}? {}".format(BName,BYear,DumVar))
    DumVar2=input('Do you want the data for the whole year? (Yes/No)')
    if ( (DumVar2=="No" or DumVar2=="no") and (DumVar=="Yes") ):
        startD=input('Enter the start date in the format DDMM like 10Jan: ')
        endD=input('Enter the end date in the format DDMM like 25Dec: ')
        date_start = startD+str(BYear)
        date_end = endD+str(BYear)
        date_st = datetime.strptime(date_start, "%d%b%Y")
        date_en = datetime.strptime(date_end, "%d%b%Y")
        print()
        print('start date:',date_st)
        print('end date:',date_en)

print()
print("Now, let's continue!")
print()
url = "https://www.ndbc.noaa.gov/view_text_file.php?filename={}h{}.txt.gz&dir=data/historical/stdmet/".format(BName.lower(),BYear)
print("Downloading the data ...")
print(url)
df_BName = pd.read_csv(url,na_values=[99,999,9999],delim_whitespace=True, skiprows=range(1,2))
print('Download is done!')


# Concatonates (#YY MM DD hh mm) into one column as DATE
df_BName.rename(columns={'#YY':'year','MM':'month','DD':'day','hh':'hour','mm':'minute'},inplace=True)
df_BName['DATE']=pd.to_datetime(df_BName[['year', 'month', 'day','hour','minute']])
df_BName.drop(columns=['year','month','day','hour','minute'],inplace=True)


if (DumVar2=="No" or DumVar2=="no"):
    df_BName=df_BName.loc[(df_BName['DATE']>=date_st) & (df_BName['DATE']<=date_en)]
    df_BName.sort_values(by='DATE',ascending=False,inplace=True)
    df_BName.reset_index(drop=True,inplace=True)
    
listLabels = ['Wind Dir. (deg)', 'Wind speed (m/s)', 'Sign. Wave Height (m)', 'Domi. Wave Period (s)',\
              'Ave. Wave Period (s)','Wave Dir. (deg)', 'Sea Level Pressure (hPa)','Air Temp. (C)', 'Sea Surface Temp. (C)',\
              'Dewpoint Temp.','Station Visib. (nautical miles)','Water Level']
df_BName.drop(columns=['GST'],inplace=True)

#print(df_BName.head())
df_BName=df_BName.dropna(axis=1, how="all")
df_BNameplt=df_BName.copy()
print(df_BNameplt.columns)
df_BNameplt.columns=['Wind Dir. (deg)', 'Wind speed (m/s)', 'Sign. Wave Height (m)', 'Domi. Wave Period (s)',
                     'Ave. Wave Period (s)','Wave Dir. (deg)', 'Sea Level Pressure (hPa)','Air Temp. (C)', 
                     'Sea Surface Temp. (C)','Date']
#print(df_BName.head())

# Now, let's plot all the data we have
print('Generating the plots ...')
print()
FNTSize = 25
for indx,clmName in enumerate(df_BNameplt.columns[:-1]):
    fig, axl1 = plt.subplots(1,1)
    fig.set_size_inches(25, 5)
    df_BNameplt.plot(kind='line',x='Date',y=clmName,fontsize=FNTSize,ax=axl1,legend=False,style='b-')
    axl1.grid()
    axl1.set_xlabel('Time',fontsize=FNTSize)
    axl1.set_ylabel(clmName,fontsize=FNTSize)
    plt.savefig('BuoyData_{}.eps'.format(df_BName.columns[indx]),format='eps',dpi=250, bbox_inches='tight')
    fig.show()

#A quick way to create new windrose axes...
def new_axes():
    fig = plt.figure(figsize=(8, 8), dpi=300, facecolor='w', edgecolor='w')
    rect = [0.1, 0.1, 0.8, 0.8]
    ax = WindroseAxes(fig, rect)
    fig.add_axes(ax)
    return ax

#...and adjust the legend box
def set_legend(ax):
    l = ax.legend()
    plt.setp(l.get_texts(), fontsize=FNTSize)
    
#FNTSize = 12
if ('WDIR' in df_BName.columns and 'WSPD' in df_BName.columns):
    theta = df_BName['WDIR'] * np.pi/180.0
    r = df_BName['WSPD']
    
    ax = new_axes()
    ax.bar(df_BName['WDIR'], r, normed=True, opening=0.8, edgecolor='white')
    set_legend(ax)
    ax.get_legend().get_title().set_fontsize('25')
    
    plt.savefig('WindRose.eps',format='eps',dpi=300, bbox_inches='tight')
    fig.show()

fig=plt.figure(figsize=(10, 8))
#FNTSize = 12
if ('MWD' in df_BName.columns and 'WVHT' in df_BName.columns):
    theta = df_BName['MWD'] * np.pi/180.0
    r = df_BName['WVHT']
    
    ax = new_axes()
    ax.bar(df_BName['MWD'], r, normed=True, opening=0.9, edgecolor='white')
    set_legend(ax)
    ax.get_legend().get_title().set_fontsize('25')
    
    plt.savefig('WaveRose.eps', format='eps',dpi=300, bbox_inches='tight')
    fig.show()
    

FNTSize = 30
if ('WVHT' in df_BName.columns and 'APD' in df_BName.columns):
    fig, (axf1, axf2) = plt.subplots(1, 2)
    fig.set_size_inches(20, 8)
    #FNTSize = 17
    # Frequency of frequencies and wave height
    df_frq = df_BName[['APD']]
    df_frq.columns=['FRQ']
    df_frq.dropna(inplace=True)
    count, bin_edges = np.histogram(df_frq)

    sigma, mu = df_frq.std(), df_frq.mean()
    mu.FRQ = round(mu.FRQ,2)
    sigma.FRQ = round(sigma.FRQ,2)
    df_frq.plot(kind='hist', bins=40 , normed=True,ax=axf1,fontsize=FNTSize,legend=False,colormap='winter')
    axf1.set_xlabel('Time Period (s)',fontsize=FNTSize)
    axf1.set_ylabel('Occurance',fontsize=FNTSize)
    axf1.text(10, 0.15, r'$\mu={},\ \sigma={}$'.format(mu.FRQ,sigma.FRQ),{'color': 'black', 'fontsize': 25, 'ha': 'left', 'va': 'center', 'bbox': dict(boxstyle="round", fc="white", ec="black", pad=0.2)})
    axf1.grid()

    sigma, mu = df_BName[['WVHT']].std(), df_BName[['WVHT']].mean()
    mu.FRQ = round(mu.WVHT,2)
    sigma.FRQ = round(sigma.WVHT,2)
    df_BName[['WVHT']].plot(kind='hist',bins=40, normed=True,ax=axf2,fontsize=FNTSize,legend=False,colormap='winter')
    axf2.set_xlabel('Ave. Wave Height (m)',fontsize=FNTSize)
    axf2.set_ylabel('Occurance',fontsize=FNTSize)
    axf2.text(4, 0.3, r'$\mu={},\ \sigma={}$'.format(mu.FRQ,sigma.FRQ),{'color': 'black', 'fontsize': 25, 'ha': 'left', 'va': 'center', 'bbox': dict(boxstyle="round", fc="white", ec="black", pad=0.2)})
    axf2.grid()
    
    plt.savefig('Occurance.eps', format='eps',dpi=300, bbox_inches='tight')
    plt.show()
    
    